<a href="https://colab.research.google.com/github/faisalnawazmir/statsInfer/blob/main/Stat_inf_1_explanatory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/data/code/

Mounted at /gdrive
/gdrive/My Drive/data/code


In [5]:
import nsfg

In [6]:
df = nsfg.ReadFemPreg()

In [7]:
df

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,cmotpreg,prgoutcome,cmprgend,flgdkmo1,cmprgbeg,ageatend,hpageend,gestasun_m,gestasun_w,wksgest,mosgest,dk1gest,dk2gest,dk3gest,bpa_bdscheck1,bpa_bdscheck2,bpa_bdscheck3,babysex,birthwgt_lb,birthwgt_oz,lobthwgt,babysex2,birthwgt_lb2,birthwgt_oz2,lobthwgt2,babysex3,birthwgt_lb3,birthwgt_oz3,lobthwgt3,cmbabdob,...,datecon_i,agecon_i,fmarout5_i,pmarpreg_i,rmarout6_i,fmarcon5_i,learnprg_i,pncarewk_i,paydeliv_i,lbw1_i,bfeedwks_i,maternlv_i,oldwantr_i,oldwantp_i,wantresp_i,wantpart_i,ager_i,fmarital_i,rmarital_i,educat_i,hieduc_i,race_i,hispanic_i,hisprace_i,rcurpreg_i,pregnum_i,parity_i,insuranc_i,pubassis_i,poverty_i,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,1093.0,NaN,1084.0,NaN,NaN,9.0,0.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,8.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1093.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,1166.0,NaN,1157.0,NaN,NaN,9.0,0.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,2.0,7.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1166.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,NaN,1.0,1156.0,NaN,1147.0,NaN,NaN,0.0,39.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,9.0,2.0,NaN,2.0,2.0,0.0,NaN,1.0,1.0,4.0,NaN,1156.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,1198.0,NaN,1189.0,NaN,NaN,0.0,39.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,2.0,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1198.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,1204.0,NaN,1195.0,NaN,NaN,0.0,39.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,2.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1204.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13588,12571,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,993.0,NaN,984.0,NaN,NaN,9.0,0.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,993.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,6.1875
13589,12571,2,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1000.0,2.0,1000.0,1.0,999.0,19.0,21.0,0.0,6.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,NaN
13590,12571,3,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1015.0,2.0,1015.0,1.0,1014.0,20.0,23.0,0.0,5.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,NaN
13591,12571,4,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,1.0,1037.0,NaN,1028.0,NaN,NaN,9.0,0.0,39.0,9.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1037.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4670.540953,5795.692880,6269.200989,1,78,NaN,7.5000


In [8]:
df.columns

Index(['caseid', 'pregordr', 'howpreg_n', 'howpreg_p', 'moscurrp', 'nowprgdk',
       'pregend1', 'pregend2', 'nbrnaliv', 'multbrth',
       ...
       'laborfor_i', 'religion_i', 'metro_i', 'basewgt', 'adj_mod_basewgt',
       'finalwgt', 'secu_p', 'sest', 'cmintvw', 'totalwgt_lb'],
      dtype='object', length=244)

In [9]:
df.columns[1]

'pregordr'

In [11]:
pregordr = df['pregordr']
type(pregordr)

pandas.core.series.Series

In [12]:
pregordr

0        1
1        2
2        1
3        2
4        3
        ..
13588    1
13589    2
13590    3
13591    4
13592    5
Name: pregordr, Length: 13593, dtype: int64

In [13]:
pregordr[0]

1

In [14]:
pregordr[2:5]

2    1
3    2
4    3
Name: pregordr, dtype: int64

In [15]:
pregordr = df.pregordr

#Variables

We have already seen two variables in the NSFG dataset, caseid and pregordr, and we
have seen that there are 244 variables in total. For the explorations in this book, I use
the following variables:

* caseid is the integer ID of the respondent.
* prglength is the integer duration of the pregnancy in weeks.
* outcome is an integer code for the outcome of the pregnancy. The code 1 indicates
a live birth.
* pregordr is a pregnancy serial number; for example, the code for a respondent’s
first pregnancy is 1, for the second pregnancy is 2, and so on.
* birthord is a serial number for live births; the code for a respondent’s first child is
1, and so on. For outcomes other than live births, this field is blank.
* birthwgt_lb and birthwgt_oz contain the pounds and ounces parts of the birth
weight of the baby.
* agepreg is the mother’s age at the end of the pregnancy.
* finalwgt is the statistical weight associated with the respondent. It is a floatingpoint
value that indicates the number of people in the U.S. population this respondent
represents.

#Transformation
agepreg contains the mother’s age at the end of the pregnancy. In the data file, agepreg
is encoded as an integer number of centiyears. So the first line divides each element of
agepreg by 100, yielding a floating-point value in years.
birthwgt_lb and birthwgt_oz contain the weight of the baby, in pounds and ounces,
for pregnancies that end in live births. In addition they use several special codes:
97 NOT ASCERTAINED
98 REFUSED
99 DON'T KNOW
Special values encoded as numbers are dangerous because if they are not handled properly, they can generate bogus results, like a 99-pound baby.The inplace flag tells replace to modify the existing Series rather than create a new
one.

In [ ]:
def CleanFemPreg(df):
  df.agepreg /= 100.0
  na_vals = [97, 98, 99]
  df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
  df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)
  df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0

As part of the IEEE floating-point standard, all mathematical operations return nan if
either argument is nan:
>>> import numpy as np
>>> np.nan / 100.0
nan

In [ ]:
#data cleaning.
# CORRECT
df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0
#Not dot notation, like this:
# WRONG!
#df.totalwgt_lb = df.birthwgt_lb + df.birthwgt_oz / 16.0

# Validation

In [16]:
df.outcome.value_counts().sort_index()

1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: outcome, dtype: int64

In [17]:
df.birthwgt_lb.value_counts(sort=False)

8.0     1889
7.0     3049
6.0     2223
4.0      229
5.0      697
10.0     132
12.0      10
14.0       3
3.0       98
1.0       40
2.0       53
0.0        8
9.0      623
11.0      26
13.0       3
15.0       1
Name: birthwgt_lb, dtype: int64

The counts for 6, 7, and 8 pounds check out, and if you add up the counts for 0-5 and
9-95, they check out, too. But if you look more closely, you will notice one value that
has to be an error, a 51 pound baby!
To deal with this error, I added a line to CleanFemPreg:

In [ ]:
  df.birthwgt_lb[df.birthwgt_lb > 20] = np.nan

#Interpretation

In [ ]:
#https://www.cdc.gov/nchs/nsfg/index.htm?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fnchs%2Fnsfg.htm

In [ ]:
#ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NSFG